In [2]:
import requests as r
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep
import concurrent.futures as cf

In [3]:
def addFolder(path, folderName):
    if path[-1] != '/':
        path += '/'
    lsFolder = "ls {} | grep '{}'".format(path, folderName)
    lsFolder = !{lsFolder}
    if len(lsFolder) == 0:
        makeFolder = 'mkdir {}{}'.format(path, folderName)
        makeFolder = !{makeFolder}
    else:
        pathExists = lsFolder[0].find('ls: cannot') == -1
        if pathExists:
            raise IOError('Folder already exists.')
        else:
            raise Exception('Path does not exist.')

def ifNoLocalFileThenDownload(path, fileName, urlPrefix):
    if path[-1] != '/':
        path += '/'
    lsFile = "ls {} | grep '{}'".format(path, fileName)
    lsFile = !{lsFile}
    if len(lsFile) == 0:
        url = '/'.join((urlPrefix, fileName))
        response = r.get(url)
        try:
            response.raise_for_status()
        except r.exceptions.HTTPError or \
               r.exceptions.ConnectionError or \
               r.exceptions.MaxRetryError or \
               NewConnectionError or \
               OSError:
            raise r.exceptions.HTTPError('Error getting file from url {}'.format(url))
        else:
            with open(path + fileName, 'wb') as f:
                f.write(response.content)
            return f'got file from {url}'
    else:
        pathExists = lsFile[0].find('ls: cannot access') == -1
        if not pathExists:
            raise Exception('Path does not exist.')
        return f'already have file {path + fileName}'

def parsePathSuffixsFromCompanyFile(companyFile, companyName, formIndex=2, formName='4'):
    pathSuffixs = []
    foundCompany = False
    for line in companyFile:
        ls = line.split()
        if len(ls) > 3:
            if ls[:formIndex] == companyName.split() and ls[formIndex] == formName:
                pathSuffixs.append(ls[-1])
                foundCompany = True
            elif foundCompany is True:
                break #sorted by company
    return pathSuffixs

In [4]:
#parameters
currYear = 2020
fileNameZip, fileName = 'company.zip', 'company.idx'
urlIndexPrefix = 'https://www.sec.gov/Archives/edgar/full-index'
urlFilePrefix = 'https://www.sec.gov/Archives'
companyName = 'BOEING CO'
formName = '4'
years = [i for i in range(1993, currYear+1)]
#years = [i for i in range(2018, 2019)] #test
quarters = ['QTR{}'.format(i+1) for i in range(4)]

In [7]:
#grab quarterly company.idx files from SEC website;
#update local archive
try:
    addFolder('./', 'data')
except:
    pass
futures = []
with cf.ThreadPoolExecutor() as executor:
    for year in years:
        #if necessary, make year folder
        try:
            addFolder('./data/', '{}'.format(year))
        except IOError as e:
            pass #folder already exists
        except Exception as e:
            print(e)
            raise #path dne
        for quarter in quarters:
            #if necessary, make quarter folder
            try:
                addFolder('./data/{}'.format(year), quarter)
            except IOError as e:
                pass #folder already exists
            except Exception as e:
                print(e)
                raise #path dne

            #if necessary, fetch company.zip from sec
            path = '/'.join(('./data', str(year), quarter, ''))
            url = '/'.join((urlIndexPrefix, str(year), quarter))                
            futures.append(executor.submit(ifNoLocalFileThenDownload, path, fileNameZip, url))
    for future in cf.as_completed(futures):
        try:
            print(future.result(), flush=True)
        except r.exceptions.HTTPError as e:
            print(e)
        except IOError as e:
            print(e)
        except Exception as e:
            print(e)
print('main loop done')

already have file ./data/2002/QTR3/company.zip
already have file ./data/1996/QTR4/company.zip
already have file ./data/2003/QTR1/company.zip
already have file ./data/2000/QTR4/company.zip
already have file ./data/2003/QTR3/company.zip
already have file ./data/2002/QTR2/company.zip
already have file ./data/1993/QTR2/company.zip
already have file ./data/1999/QTR4/company.zip
already have file ./data/1994/QTR1/company.zip
already have file ./data/2004/QTR1/company.zip
already have file ./data/2000/QTR3/company.zip
already have file ./data/1996/QTR1/company.zip
already have file ./data/2010/QTR1/company.zip
already have file ./data/2009/QTR4/company.zip
already have file ./data/2013/QTR2/company.zip
already have file ./data/2000/QTR2/company.zip
already have file ./data/2013/QTR4/company.zip
already have file ./data/1999/QTR2/company.zip
already have file ./data/2013/QTR1/company.zip
already have file ./data/2004/QTR4/company.zip
already have file ./data/2002/QTR1/company.zip
already have 

In [ ]:
#extract company.idx files
for year in years:
    for quarter in quarters:
        path = '/'.join(('./data', str(year), quarter, ''))
        unzipFile = 'unzip {} -d {}'.format(path + fileNameZip, path)
        unzipFile = !{unzipFile}
        if len(unzipFile) == 3:
            print('extracted file already exists')
        else:
            unzipSuccessful = unzipFile[0].find('unzip:') == -1
            if not unzipSuccessful:
                print('File {} unzip failed:\n\t{}'.format(path + fileNameZip, unzipFile))
            else:
                print('extracted file from {} to {}'.format(path + fileNameZip, 
                                                            unzipFile[1][unzipFile[1].find(':')+2:]))

In [8]:
#grab form 4's for company companyName from SEC website
#as indexed by company.idx in year/quarter/ local files;
#update local archive
futures = []
with cf.ThreadPoolExecutor() as executor:
    for year in years:
        for quarter in quarters:
            path = '/'.join(('./data', str(year), quarter, ''))
            try:
                with open(path + fileName, 'r') as f:
                    pathSuffixs = parsePathSuffixsFromCompanyFile(f, companyName, formName=formName)
            except FileNotFoundError:
                print('no file {} found'.format(path + fileName))
            else:
                #if necessary ,make req'd directory
                try:
                    pss = pathSuffixs[0].split(sep='/')
                except IndexError:
                    pass
                else:
                    try:
                        addFolder('./data/', pss[2])
                    except:
                        pass

                    #if necessary, fetch files from sec
                    for pathSuffix in pathSuffixs:
                        #pathSuffix form: 'edgar/data/.../....txt'
                        pss = pathSuffix.split(sep='/')
                        path = '/'.join(('.', *pss[1:-1], ''))
                        textFileName = pss[-1]
                        url = '/'.join((urlFilePrefix, *pss[:-1]))
                        futures.append(executor.submit(ifNoLocalFileThenDownload, path, textFileName, url))
    for future in cf.as_completed(futures):
        try:
            print(future.result(), flush=True)
        except r.exceptions.HTTPError as e:
            print(e)
        except IOError as e:
            print(e)
        except Exception as e:
            print(e)
print('main loop done')

no file ./data/2020/QTR3/company.idx found
no file ./data/2020/QTR4/company.idx found
already have file ./data/12927/0001225208-15-007474.txt
already have file ./data/12927/0001225208-15-007473.txt
already have file ./data/12927/0001225208-15-009488.txt
already have file ./data/12927/0001225208-15-015196.txt
already have file ./data/12927/0001225208-15-006638.txt
already have file ./data/12927/0001225208-15-009491.txt
already have file ./data/12927/0001225208-15-015195.txt
already have file ./data/12927/0000891020-03-001340.txt
already have file ./data/12927/0001225208-15-006637.txt
already have file ./data/12927/0001225208-15-020264.txt
already have file ./data/12927/0001225208-15-009492.txt
already have file ./data/12927/0001225208-15-006636.txt
already have file ./data/12927/0001225208-15-009486.txt
already have file ./data/12927/0001225208-15-020265.txt
already have file ./data/12927/0001137638-02-000018.txt
already have file ./data/12927/0001225208-15-006635.txt
already have file 

already have file ./data/12927/0001137638-05-000324.txt
already have file ./data/12927/0001225208-09-000783.txt
already have file ./data/12927/0001137638-05-000078.txt
already have file ./data/12927/0001137638-07-000119.txt
already have file ./data/12927/0001137638-07-000136.txt
already have file ./data/12927/0001137638-07-000139.txt
already have file ./data/12927/0001137638-05-000082.txt
already have file ./data/12927/0001137638-04-000699.txt
already have file ./data/12927/0000891020-03-000133.txt
already have file ./data/12927/0001137638-07-000118.txt
already have file ./data/12927/0001137638-07-000141.txt
already have file ./data/12927/0001137638-05-000065.txt
already have file ./data/12927/0001137638-05-000453.txt
already have file ./data/12927/0001137638-07-000117.txt
already have file ./data/12927/0001137638-07-000140.txt
already have file ./data/12927/0001137638-05-000321.txt
already have file ./data/12927/0001137638-07-000116.txt
already have file ./data/12927/0001137638-05-000

already have file ./data/12927/0001225208-14-009996.txt
already have file ./data/12927/0001225208-16-036330.txt
already have file ./data/12927/0001225208-12-020850.txt
already have file ./data/12927/0001225208-11-023028.txt
already have file ./data/12927/0001137638-06-000227.txt
already have file ./data/12927/0001225208-14-021153.txt
already have file ./data/12927/0001137638-04-000728.txt
already have file ./data/12927/0001137638-06-000549.txt
already have file ./data/12927/0001225208-09-004864.txt
already have file ./data/12927/0001225208-12-006902.txt
already have file ./data/12927/0001137638-06-000040.txt
already have file ./data/12927/0001137638-06-000329.txt
already have file ./data/12927/0001225208-09-009860.txt
already have file ./data/12927/0001225208-16-027906.txt
already have file ./data/12927/0001225208-16-032695.txt
already have file ./data/12927/0001225208-14-008068.txt
already have file ./data/12927/0001225208-14-009999.txt
already have file ./data/12927/0001225208-12-015

already have file ./data/12927/0001137638-06-000127.txt
already have file ./data/12927/0001137638-05-000507.txt
already have file ./data/12927/0001225208-10-016693.txt
already have file ./data/12927/0001137638-05-000648.txt
already have file ./data/12927/0001137638-05-000313.txt
already have file ./data/12927/0001225208-11-017620.txt
already have file ./data/12927/0001225208-10-011303.txt
already have file ./data/12927/0001137638-06-000291.txt
already have file ./data/12927/0001137638-05-000660.txt
already have file ./data/12927/0001225208-10-016696.txt
already have file ./data/12927/0001137638-05-000304.txt
already have file ./data/12927/0001225208-11-011209.txt
already have file ./data/12927/0001225208-11-000699.txt
already have file ./data/12927/0001225208-10-016699.txt
already have file ./data/12927/0001137638-05-000312.txt
already have file ./data/12927/0001137638-05-000508.txt
already have file ./data/12927/0001137638-05-000305.txt
already have file ./data/12927/0001225208-10-022

already have file ./data/12927/0001225208-16-023776.txt
already have file ./data/12927/0000891020-02-001429.txt
already have file ./data/12927/0001225208-15-005777.txt
already have file ./data/12927/0001225208-15-005776.txt
already have file ./data/12927/0001225208-16-031045.txt
already have file ./data/12927/0001225208-15-005775.txt
already have file ./data/12927/0001225208-16-027890.txt
already have file ./data/12927/0001225208-15-005774.txt
already have file ./data/12927/0001225208-16-023777.txt
already have file ./data/12927/0001225208-15-005771.txt
already have file ./data/12927/0001225208-16-031046.txt
already have file ./data/12927/0001225208-15-005769.txt
already have file ./data/12927/0001225208-16-040190.txt
already have file ./data/12927/0001225208-16-027888.txt
already have file ./data/12927/0001225208-15-005768.txt
already have file ./data/12927/0001225208-16-037459.txt
already have file ./data/12927/0001225208-16-037458.txt
already have file ./data/12927/0001225208-15-005

...

already have file ./data/12927/0001137638-06-000378.txt
already have file ./data/12927/0001137638-06-000553.txt
already have file ./data/12927/0001137638-04-000164.txt
already have file ./data/12927/0001137638-06-000524.txt
already have file ./data/12927/0001137638-04-000427.txt
already have file ./data/12927/0001137638-04-000431.txt
already have file ./data/12927/0001137638-06-000377.txt
already have file ./data/12927/0001137638-04-000163.txt
already have file ./data/12927/0001137638-07-000024.txt
already have file ./data/12927/0001137638-06-000372.txt
already have file ./data/12927/0001137638-04-000162.txt
already have file ./data/12927/0001137638-06-000552.txt
already have file ./data/12927/0001137638-04-000616.txt
already have file ./data/12927/0001137638-06-000371.txt
already have file ./data/12927/0001137638-04-000161.txt
already have file ./data/12927/0001137638-04-000615.txt
already have file ./data/12927/0001225208-13-015799.txt
already have file ./data/12927/0001225208-13-011

got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005127.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005016.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005125.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005017.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005126.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005019.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005018.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005015.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005133.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005128.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-17-005135.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/00012

got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000723.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000720.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000719.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000715.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-002579.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000724.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000725.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-000732.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-004133.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-004144.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-18-004471.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/00012

got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000590.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000589.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000588.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000593.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000591.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000592.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000594.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000595.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000597.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000598.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-19-000600.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/00012

got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000560.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000562.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000563.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000564.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000565.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000566.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000567.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000570.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000569.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000574.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/0001225208-20-000571.txt
got file from https://www.sec.gov/Archives/edgar/data/12927/00012

In [ ]:
#clean up company.idx files
for year in years:
    for quarter in quarters:
        path = '/'.join(('./data', str(year), quarter, ''))
        cleanUpFile = 'rm {}'.format(path + fileName)
        cleanUpFile = !{cleanUpFile}
        cleanUpSuccessful = len(cleanUpFile) == 0
        if not cleanUpSuccessful:
            print('File {} does not exist'.format(path + fileName))
        else:
            print('cleaned file at {}'.format(path + fileName))